Ce notebook contient les différentes fonctions utilisées pour la mise en œuvre de l'algorithme LBG, également connu sous le nom d'algorithme de Lloyd généralisé. 

On commence par importer les différentes librairies nécessaires :

In [53]:
import math
from functools import reduce
from collections import defaultdict
import random

data_size = 0
dim = 0


Les fonctions implémentées sont les suivantes :

genCodebook : génère un codebook (ensemble de vecteurs de taille N) à partir d'un ensemble de vecteurs d'apprentissage donné.  On itère jusqu'à ce que a convergence soit atteinte.

In [54]:
def genCodebook(data, N, delta=0.00001): #delta = seuil du max de distortion moyenne
    global data_size
    global dim
    
    data_size = len(data)
    assert data_size > 0
    
    dim = len(data[0])
    assert dim > 0
    
    codebook = []
    codebook_poids_abs = [data_size]
    codebook_poids_rel = [1.0]
    
    # Calcul du vecteur initial du codebook : on prend la moyenne de tous les vecteurs de notre image
    
    c0 = moy_vec_de_vecs(data, dim, data_size)
    codebook.append(c0)
    
    # Calcul de la distorsion moyenne
    
    dist_moy = distortion_moy_c0(c0, data)
    
    # On split les "codevectors" autant que nécessaire
    
    while len(codebook) < N:
        codebook, codebook_poids_abs, codebook_poids_rel, dist_moy = split_codebook(data, codebook, dist_moy, delta)
    return codebook, codebook_poids_abs, codebook_poids_rel

moy_vec_de_vecs : calcule la moyenne de tous les vecteurs d'un ensemble de vecteurs donné.

In [55]:
def moy_vec_de_vecs(vecs, dim_vecs=None, taille_vecs=None):
    
    dim_vecs = dim_vecs or len(vecs[0])
    taille_vecs = taille_vecs or len(vecs)
    vec_moy = [0.0] * dim_vecs  
    for vec in vecs:
        for i, x in enumerate(vec):
            vec_moy[i] += x / taille_vecs

    return vec_moy

distortion_moy_c0 : calcule la distorsion moyenne d'un ensemble de vecteurs par rapport à un vecteur donné.

In [56]:
def distortion_moy_c0(c0, data, size=None):

    size = size or data_size
    return reduce(lambda s, d:  s + d / size,
                  (distance_eucli(c0, vec)
                   for vec in data),
                  0.0)

distortion_moy_codevector_list : calcule la distorsion moyenne d'un ensemble de vecteurs par rapport à un codebook donné.

In [57]:
def distortion_moy_codevector_list(codevector_list, data):
    return reduce(lambda s, d: s + d / data_size,
                  (distance_eucli(codevector, vec)
                   for codevector, vec in zip(codevector_list, data)),
                  0.0)

distance_eucli : calcule la distance euclidienne entre deux vecteurs.

In [58]:
def distance_eucli(vec1, vec2):
    return math.sqrt(sum((x1 - x2) ** 2 for x1, x2 in zip(vec1, vec2)))

création du vecteur aléatoire epsilon : 

In [59]:
def gen_vec_epsilon(codevector):
    epsilon = []
    for i in range(len(codevector)):
        epsilon.append(random.uniform(-0.00001, 0.00001))
    return epsilon

split_codebook : divise un codebook en deux parties et renvoie le codebook mis à jour.

In [60]:
def split_codebook(data, codebook, moy_dist_initial, delta=0.00001):
   
    # split codevectors
    new_codevectors = []
    for codevector in codebook:
        epsilon = gen_vec_epsilon(codevector)
        # On crée les deux nouveaux codevectors en adjutant et en soustrayant epsilon à chaque composante du codevector
        c1 = [cv + ep for cv, ep in zip(codevector, epsilon)]
        c2 = [cv - ep for cv, ep in zip(codevector, epsilon)]
        new_codevectors.extend((c1, c2))

    codebook = new_codevectors
    len_codebook = len(codebook)
    poids_abs = [0] * len_codebook
    poids_rel = [0.0] * len_codebook
    
    # print('> split jusqu'à la bonne taille', len_codebook)

    # on cherche la convergence en minimisant la distorsion moyenne
    moy_dist = 0
    err = delta + 1
    num_iter = 0
    while err > delta:
        # On cherche les codevectors les plus proches pour chaque vecteur de données (on trouve la proximité de chaque codevector)
        plus_proche_codevector_list = [None] * data_size
        dist_moy = distortion_moy_codevector_list(plus_proche_codevector_list, data)# liste contenant le vecteur le plus poche pour chaque vecteur de données
        vecs_proche_codevector = defaultdict(list)         # On mappe les vecteurs de données
        vec_idxs_proche_codevector = defaultdict(list)     # de même
        for i, vec in enumerate(data):  # pour chaque vecteur en entrée
            dist_min = None
            plus_proche_codevector_index = None
            for i_c, c in enumerate(codebook):  # pour chaque codevector
                d = distance_eucli(vec, c)
                if dist_min is None or d < dist_min:    # Trouver le nouveau codevector le plus proche
                    dist_min = d
                    plus_proche_codevector_list[i] = c
                    plus_proche_codevector_index = i_c
            vecs_proche_codevector[plus_proche_codevector_index].append(vec)
            vec_idxs_proche_codevector[plus_proche_codevector_index].append(i)

        # on met à jour le codebook : on recalcule chaque codevector pour qu'il se trouve au centre des points de proximité
        for i_c in range(len_codebook): # pour chaque index de codevector
            vecs = vecs_proche_codevector.get(i_c) or []   # obtenir les vecteurs proches de ce codevector
            num_vecs_proche_codevector = len(vecs)
            if num_vecs_proche_codevector > 0:
                new_codevector = moy_vec_de_vecs(vecs, dim)     # On calcule le nouveau centre de ce codevector
                codebook[i_c] = new_codevector                   # nouveau codevector dans codebook
                for i in vec_idxs_proche_codevector[i_c]:          # update in input vector index -> codevector mapping list
                    plus_proche_codevector_list[i] = new_codevector

                # update the weights
                poids_abs[i_c] = num_vecs_proche_codevector
                poids_rel[i_c] = num_vecs_proche_codevector / data_size

        # On calcule de nouveau la distorsion moyenne
        dist_moy_prec = dist_moy if dist_moy > 0 else moy_dist_initial
        dist_moy = distortion_moy_codevector_list(plus_proche_codevector_list, data)

        # On calcule de nouveau l'erreur
        err = (dist_moy_prec - dist_moy) / dist_moy_prec
        print(plus_proche_codevector_list)
        print('> iteration', num_iter, 'dist_moy', dist_moy, 'dist_moy_prec', dist_moy_prec, 'err', err)

        num_iter += 1

    return codebook, poids_abs, poids_rel, dist_moy

Test :

In [61]:
testdata = [(-1.5, 2.0, 5.0),
            (-2.0, -2.0, 0.0),
            (1.0, 1.0, 2.0),
            (1.5, 1.5, 1.2),
            (1.0, 2.0, 5.6),
            (1.0, -2.0, -2.0),
            (1.0, -3.0, -2.0),
            (1.0, -2.5, -4.5)]

for codebook_size in (1, 2, 4, 8):
    print('Génération du codebook', codebook_size)
    codebook, codebook_p_abs, codebook_p_rel = genCodebook(testdata, codebook_size, delta=0.00001)
    print('output:')
    for i, c in enumerate(codebook):
        print('> %s, poids_abs=%d, poids_rel=%f' % (codebook, codebook_p_abs[i], codebook_p_rel[i]))

Génération du codebook 1
output:
> [[0.375, -0.375, 0.6624999999999999]], poids_abs=8, poids_rel=1.000000
Génération du codebook 2


TypeError: 'NoneType' object is not iterable